In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:     path_main  = file.read()
with open('../path_gwls.txt', 'r') as file:     path_gwls  = file.read()
with open('../path_EUR-11.txt', 'r') as file:   path_eur11 = file.read()
    
dir_CORDEX   = f'{path_main}Data/EURO-CORDEX/HSIs/'
dir_HSIstats = f'{path_main}Data/EURO-CORDEX/HSI_stats/'
dir_GWL      = f'{path_gwls}cmip5_all_ens/'
dir_EMT      = f'{path_main}Data/EURO-CORDEX/EMT/'
dir_names    = f'{path_main}Scripts/Model_lists/'
dir_scripts  = f'{path_main}/Scripts/'
dir_sftlf    = f'{path_eur11}/historical/sftlf/'
dir_out      = f'{path_main}Data/Plot_preparation/Threshold_Exceedance/'
if not os.path.exists(dir_out): os.mkdir(dir_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade', 'Zagreb',
          'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam', 'London', 'Dublin',
          'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk', 'Vilnius', 'Riga', 'Moscow',
          'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)
    
#Define HSIs
HSI_names = ['TN', 'TX']

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp26'] = []
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP26.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp26'].append(eval(line[:-1]))
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))

#Read warming levels
fname = dir_GWL + 'cmip5_warming_levels_all_ens_1850_1900_no_bounds_check.yml'
with open(fname, 'r') as file:
    GWL_data = yaml.safe_load(file)

#Define warming levels
GWL_levels = ['1981-2010', '10', '20', '30']

#Define out_name
out_name = '_3x3'
# out_name = ''


## Define thresholds

In [ ]:
#Level 1: warm (caution)
#Level 2: hot (extreme caution)
#Level 3: very_hot (danger)
#Level 4: sweltering (extreme danger)
thresholds = {}

#Tropical nights (and sensitivity tests)
thresholds["TN_Level1"] = 15
thresholds["TN_Level2"] = 17
thresholds["TN_Level3"] = 20
thresholds["TN_Level4"] = 23

#TX
thresholds["TX_Level1"] = 25
thresholds["TX_Level2"] = 27
thresholds["TX_Level3"] = 30
thresholds["TX_Level4"] = 33

#Names for NetCDF file
threshold_names = ["Level1", "Level2", "Level3", "Level4"]


## Read land-sea mask

In [ ]:
N = 3

#Initialize dictionary
data_LSM = dict()

#Loop over all models
for model in all_models['rcp85']:
    
    #Get file names
    fnames = [file for file in os.listdir(dir_sftlf) if model[0] in file and model[1] in file]
    
    #Define file names for certain models, for which no LSM exists
    if model[1]=='IPSL-WRF381P':
        fnames = ['sftlf_EUR-11_CNRM-CERFACS-CNRM-CM5_historical_r0i0p0_IPSL-WRF381P_v2-ADAPTED-FROM-OLDER-VERSION_fx.nc']
    if model[0]=='ICHEC-EC-EARTH' and model[1]=='CLMcom-ETH-COSMO-crCLIM-v1-1':
        fnames = ['sftlf_EUR-11_MPI-M-MPI-ESM-LR_historical_r0i0p0_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_fx.nc']
    if model[0]=='IPSL-IPSL-CM5A-MR' and model[1]=='DMI-HIRHAM5':
        fnames = ['sftlf_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_DMI-HIRHAM5_v1_fx.nc']    
    
    #Get file name
    if len(fnames)>=1:
        fname = fnames[0]
    else:
        sys.exit('File name not defined')
    
    #Read data
    data_sftlf = xr.open_dataset(dir_sftlf + fname)
 
    #Convert LSM data to %
    if data_sftlf.sftlf.max()<50:
        data_sftlf['sftlf'] = 100 * data_sftlf['sftlf']

    #Convert longitude from [0, 360] to [-180, 180]
    if 'longitude' in data_sftlf.coords:  lat_name, lon_name = 'latitude', 'longitude'
    elif 'lon' in data_sftlf.coords:      lat_name, lon_name = 'lat', 'lon'
    if data_sftlf[lon_name].max()>180:
        data_sftlf[lon_name] = data_sftlf[lon_name].where(data_sftlf[lon_name]<180, ((data_sftlf[lon_name] + 180) % 360) - 180)
    
    #Loop over cities
    for city in cities:
        
        #Get lat and lon of city
        lat_sel, lon_sel = city_coords[city]

        #Find grid point closest to city
        loc_city = (np.abs(data_sftlf[lon_name] - lon_sel)) + (np.abs(data_sftlf[lat_name] - lat_sel))
        ind_city = np.unravel_index(np.argmin(loc_city.values), loc_city.shape)

        #Select NxN box around grid point
        N1 = int(N/2 - 0.5)
        N2 = int(N/2 + 0.5)
        lat_rng  = slice(ind_city[0] - N1, ind_city[0] + N2)
        lon_rng  = slice(ind_city[1] - N1, ind_city[1] + N2)
        if 'rlat' in data_sftlf.dims:   data_sftlf_city = data_sftlf.isel(rlat=lat_rng, rlon=lon_rng)
        elif 'x' in data_sftlf.dims:    data_sftlf_city = data_sftlf.isel(y=lat_rng, x=lon_rng)
        
        #Save in dictionary
        data_LSM[model[0] + '_' + model[1] + '_' + city] = data_sftlf_city.sftlf


## Prepare data according to European warming (EMT)

In [ ]:
#Define output folder
dir_save = dir_out + 'EURO-CORDEX' + out_name + '/'

#Define RCP
RCP = 'rcp85'
scenarios = ['historical', RCP]

#Define changes in EMT relative to 1981-2010
EMT_change     = np.array([0.0, 1.0, 2.0, 3.0])
EMT_change_str = ['1981-2010', '1.0K', '2.0K', '3.0K']

#Define threshold levels
THR_levels = ['Level1', 'Level2', 'Level3', 'Level4']

#Define transformations to apply
transformations = ['no_trans', 'trans_mean', 'trans_z']

#Define minimum fraction of land required on each grid point
land_fractions = [0, 25, 50, 75, 100]

#Combine model names to one string
mod_names = ["_".join(model) for model in all_models[RCP]]

#Create empty numpy array and define coordinates
if out_name=='_3x3':
    da_empty = np.zeros((len(all_models[RCP]), len(cities), len(EMT_change), len(THR_levels), len(transformations), len(land_fractions), 2)) * np.NaN
    coords={'model':          ('model', mod_names),
            'city':           ('city', cities),
            'EMT_change':     ('EMT_change', EMT_change_str),
            'THR_level':      ('THR_level', THR_levels),
            'transformation': ('transformation', transformations),
            'land_frac':      ('land_frac', land_fractions),
            'range_3x3box':   ('range_3x3box', ['min', 'max'])}
else:
    da_empty = np.zeros((len(all_models[RCP]), len(cities), len(EMT_change), len(THR_levels), len(transformations))) * np.NaN
    coords={'model':          ('model', mod_names),
            'city':           ('city', cities),
            'EMT_change':     ('EMT_change', EMT_change_str),
            'THR_level':      ('THR_level', THR_levels),
            'transformation': ('transformation', transformations)}

#Create empty data array for storing SREX-averaged heat indices
data_coll = xr.Dataset(coords=coords)
for HSI in HSI_names:  data_coll[HSI] = (coords.keys(), da_empty.copy())

#Loop over cities
for city in cities:
    
    print(city, end=', ')

    #Loop over models
    for i1, model in enumerate(all_models[RCP]):

        #Get name of CMIP5 driving model
        if ('CNRM-CERFAC' in model[0]) or ('CSIRO-QCCCE' in model[0]) or ('MPI-M' in model[0]) or ('NOAA-GFDL' in model[0]):
            mod_CMIP5 = '-'.join(model[0].split('-')[2:])
        else:
            mod_CMIP5 = '-'.join(model[0].split('-')[1:])
            
        #Loop over scenarios
        create = 1
        for scen in scenarios:

            #Define folder
            dir_data = dir_CORDEX + city + '/' + scen + '/'

            #Read data
            fnames_CORDEX = [file for file in os.listdir(dir_data) if scen + '_' in file and model[0] in file and model[1] in file and model[2] in file]
            if out_name=='':  fnames_CORDEX = [file for file in fnames_CORDEX if 'HSIs_' + city in file]
            else:             fnames_CORDEX = [file for file in fnames_CORDEX if out_name[1::] in file]
            if len(fnames_CORDEX)!=1:  sys.exit('File is not unique')
            data_read = xr.open_dataset(dir_data + fnames_CORDEX[0])

            #Concatenate data
            if create==1:
                data_CORDEX = data_read
                create = 0
            else:
                data_CORDEX = xr.concat((data_CORDEX, data_read), dim='time')
        
        #Read European mean temperature (EMT)
        files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if "_".join(model) in file and 'EMT_' in file])
        data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')

        #Calculate EMT relative to 1981-2010 and calculate 20-year means
        dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
        dataEMT_rel = data_EMT - dataEMT_ref
        dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()

        #Loop over selected EMT levels
        for dEMT, dEMT_str in zip(EMT_change[1::], EMT_change_str[1::]):

            #Identify 20-year period in wich level is reached for first time
            ind  = np.where(dataEMT_20y.tas>dEMT)[0][0]
            central_year = dataEMT_20y.isel(time=ind).time.dt.year
            start_year   = int(central_year - 20 / 2)
            end_year     = int(central_year + (20 / 2 - 1))
            years_sel    = slice(str(start_year), str(end_year))

            if end_year>2099:
                print(model)
                print(end_year)

            #Get data in 20y-period
            data_ref = data_CORDEX.sel(time=slice('1981', '2010'))
            data_20y = data_CORDEX.sel(time=years_sel)

            #Loop over HSIs
            for HSI in HSI_names:

                data_trans_ref = dict()
                data_trans_20y = dict()
                
                #Read data for z-tranformation
                fname_stats = dir_HSIstats + HSI + '/' + HSI + '_mean_std_JJA_' + "_".join(model) + "_JJA_1981-2010.csv"
                data_mu_sig = pd.read_csv(fname_stats, index_col=0)
                
                #Extract data
                mu_CORDEX  = data_mu_sig.loc['mu_model', city]
                sig_CORDEX = data_mu_sig.loc['sigma_model', city]
                mu_ERA5L   = data_mu_sig.loc['mu_ERA5L', city] - 273.15 * (data_mu_sig.loc['mu_ERA5L', city]>150)
                sig_ERA5L  = data_mu_sig.loc['sigma_ERA5L', city]

                #transformation to ERA5 mean (and variability)
                z_mod_ref = (data_ref[HSI] - mu_CORDEX) / sig_CORDEX
                z_mod_20y = (data_20y[HSI] - mu_CORDEX) / sig_CORDEX
                data_trans_ref['trans_mean'] = data_ref[HSI] - mu_CORDEX + mu_ERA5L
                data_trans_20y['trans_mean'] = data_20y[HSI] - mu_CORDEX + mu_ERA5L
                data_trans_ref['trans_z']    = z_mod_ref * sig_ERA5L + mu_ERA5L
                data_trans_20y['trans_z']    = z_mod_20y * sig_ERA5L + mu_ERA5L

                #Correct TN and TX
                if HSI in ['TN', 'TX']:
                    data_trans_ref['no_trans'] = data_ref[HSI] - 273.15
                    data_trans_20y['no_trans'] = data_20y[HSI] - 273.15
                else:
                    data_trans_ref['no_trans'] = data_ref[HSI]
                    data_trans_20y['no_trans'] = data_20y[HSI]
                    
                #Loop over levels'time'
                for THR_level in THR_levels:
                    
                    for transformation in transformations:
                    
                        #Calculate exceedance
                        data_exc_ref = data_trans_ref[transformation]>=thresholds[HSI + "_" + THR_level]
                        data_exc_20y = data_trans_20y[transformation]>=thresholds[HSI + "_" + THR_level]
                        data_exc_ref = data_exc_ref.sum('time') / (end_year - start_year + 1)
                        data_exc_20y = data_exc_20y.sum('time') / (end_year - start_year + 1)
                        
                        if out_name=='_3x3':
                            
                            #Read land fraction for each model and city
                            land_frac = data_LSM[model[0] + '_' + model[1] + '_' + city]

                            #Rename land fraction for MPI-WRF combination
                            if (model[0]=='MPI-M-MPI-ESM-LR') and (model[1]=='IPSL-WRF381P'):
                                land_frac = land_frac.rename({'rlat': 'y', 'rlon': 'x'})                       

                            #Get coordinate names
                            if 'longitude' in land_frac.coords:  lat_name1, lon_name1 = 'latitude', 'longitude'
                            elif 'lon' in land_frac.coords:      lat_name1, lon_name1 = 'lat', 'lon'                        
                            if 'rlon' in land_frac.coords:       lat_name2, lon_name2 = 'rlat', 'rlon'
                            elif 'x' in land_frac.coords:        lat_name2, lon_name2 = 'y', 'x'   

                            #Check if coordinates agree and reindex if they are slightly shifted
                            check1a = np.sum(np.abs(land_frac[lat_name1].values - data_exc_ref[lat_name1].values))
                            check1b = np.sum(np.abs(land_frac[lon_name1].values - data_exc_ref[lon_name1].values))
                            check2a = np.sum(np.abs(land_frac[lat_name2].values - data_exc_ref[lat_name2].values))
                            check2b = np.sum(np.abs(land_frac[lon_name2].values - data_exc_ref[lon_name2].values))
                            if (check1a>0.001) or (check1b>0.001):
                                sys.exit('Coordinates between data and LSM do not agree')
                            if (check2a!=0) or (check2b!=0):
                                land_frac[lon_name2] = data_exc_ref[lon_name2]
                                land_frac[lat_name2] = data_exc_ref[lat_name2]                  

                            #Loop over different minimum land fractions
                            for land_min in land_fractions:

                                #Apply land-sea mask
                                data_exc_ref_land = data_exc_ref.where(land_frac>=land_min)
                                data_exc_20y_land = data_exc_20y.where(land_frac>=land_min)

                                #Test that final array still has all entries
                                if data_exc_ref_land.shape!=(3,3):  sys.exit('bla')

                                #Put data in array
                                data_coll[HSI].loc[{"model": "_".join(model), "city": city, "EMT_change": '1981-2010', "THR_level": THR_level, "transformation": transformation, "land_frac": land_min, 'range_3x3box': 'min'}] = data_exc_ref_land.min()
                                data_coll[HSI].loc[{"model": "_".join(model), "city": city, "EMT_change": '1981-2010', "THR_level": THR_level, "transformation": transformation, "land_frac": land_min, 'range_3x3box': 'max'}] = data_exc_ref_land.max()
                                data_coll[HSI].loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "THR_level": THR_level, "transformation": transformation, "land_frac": land_min, 'range_3x3box': 'min'}]    = data_exc_20y_land.min()
                                data_coll[HSI].loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "THR_level": THR_level, "transformation": transformation, "land_frac": land_min, 'range_3x3box': 'max'}]    = data_exc_20y_land.max()
                        else:
                            
                            #Put data in array
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "EMT_change": '1981-2010', "THR_level": THR_level, "transformation": transformation}] = data_exc_ref
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "EMT_change": dEMT_str, "THR_level": THR_level, "transformation": transformation}]    = data_exc_20y
                    
#Save data in file
fname_out = dir_save + 'HSIs-ThresholdExceedance_' + RCP + '_EMT.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll.to_netcdf(fname_out)


## Prepare GWL data

In [ ]:
#Define output folder
dir_save = dir_out + 'EURO-CORDEX' + out_name + '/'

#Define RCP
RCP = 'rcp85'
scenarios = ['historical', RCP]

THR_levels = ['Level1', 'Level2', 'Level3', 'Level4']

#Define minimum fraction of land required on each grid point
land_fractions = [0, 25, 50, 75, 100]

#Combine model names to one string
mod_names = ["_".join(model) for model in all_models[RCP]]

#Create empty numpy array and define coordinates
if out_name=='_3x3':
    da_empty = np.zeros((len(all_models[RCP]), len(cities), len(GWL_levels), len(THR_levels), len(land_fractions), 2)) * np.NaN
    coords={'model':        ('model', mod_names),
            'city':         ('city', cities),
            'GWL_level':    ('GWL_level', GWL_levels),
            'THR_level':    ('THR_level', THR_levels),
            'land_frac':    ('land_frac', land_fractions),
            'range_3x3box': ('range_3x3box', ['min', 'max'])}
else:
    da_empty = np.zeros((len(all_models[RCP]), len(cities), len(GWL_levels), len(THR_levels))) * np.NaN
    coords={'model':     ('model', mod_names),
            'city':      ('city', cities),
            'GWL_level': ('GWL_level', GWL_levels),
            'THR_level': ('THR_level', THR_levels)}

#Create empty data array for storing SREX-averaged heat indices
data_coll = xr.Dataset(coords=coords)
for HSI in HSI_names:  data_coll[HSI] = (coords.keys(), da_empty.copy())

#Loop over cities
for city in cities:
    
    print(city, end=', ')

    #Loop over models
    for i1, model in enumerate(all_models[RCP]):

        #Get name of CMIP5 driving model
        if ('CNRM-CERFAC' in model[0]) or ('CSIRO-QCCCE' in model[0]) or ('MPI-M' in model[0]) or ('NOAA-GFDL' in model[0]):
            mod_CMIP5 = '-'.join(model[0].split('-')[2:])
        else:
            mod_CMIP5 = '-'.join(model[0].split('-')[1:])

        #Loop over scenarios
        create = 1
        for scen in scenarios:

            #Define folder
            dir_data = dir_CORDEX + city + '/' + scen + '/'

            #Read data
            fnames_CORDEX = [file for file in os.listdir(dir_data) if scen + '_' in file and model[0] in file and model[1] in file and model[2] in file]
            if out_name=='':  fnames_CORDEX = [file for file in fnames_CORDEX if 'HSIs_' + city in file]
            else:             fnames_CORDEX = [file for file in fnames_CORDEX if out_name[1::] in file]
            if len(fnames_CORDEX)!=1:  sys.exit('File is not unique')
            data_read = xr.open_dataset(dir_data + fnames_CORDEX[0])

            #Concatenate data
            if create==1:
                data_CORDEX = data_read
                create = 0
            else:
                data_CORDEX = xr.concat((data_CORDEX, data_read), dim='time')            
        
        #Read time periods when certain global warming levels (GWL) are reached
        time_GWL = dict()
        for GWL_level in GWL_levels[1::]:
            data_level = GWL_data['warming_level_' + GWL_level]
            entry_sel = [entry for entry in data_level if entry['model']==mod_CMIP5 and entry['exp']==RCP and entry['ensemble']==model[2]]

            #Select data in time period when GWL is reached
            start_year = entry_sel[0]['start_year']
            end_year   = entry_sel[0]['end_year']
            time_GWL = slice(str(entry_sel[0]['start_year']), str(entry_sel[0]['end_year']))

            #Get data in 20y-period
            data_ref = data_CORDEX.sel(time=slice('1981', '2010'))
            data_20y = data_CORDEX.sel(time=time_GWL)
            
            #Loop over HSIs
            for HSI in HSI_names:
                
                #Correct TN and TX
                if HSI in ['TN', 'TX']:
                    data_sel_ref = data_ref[HSI] - 273.15
                    data_sel_20y = data_20y[HSI] - 273.15
                else:
                    data_sel_ref = data_ref[HSI]
                    data_sel_20y = data_20y[HSI]
                
                #Loop over levels
                for THR_level in THR_levels:
                    
                    #Calculate exceedance
                    data_exc_ref = data_sel_ref>=thresholds[HSI + "_" + THR_level]
                    data_exc_20y = data_sel_20y>=thresholds[HSI + "_" + THR_level]
                    data_exc_ref = data_exc_ref.sum('time') / (end_year - start_year + 1)
                    data_exc_20y = data_exc_20y.sum('time') / (end_year - start_year + 1)

                    if out_name=='_3x3':
                        
                        #Read land fraction for each model and city
                        land_frac = data_LSM[model[0] + '_' + model[1] + '_' + city]

                        #Rename land fraction for MPI-WRF combination
                        if (model[0]=='MPI-M-MPI-ESM-LR') and (model[1]=='IPSL-WRF381P'):
                            land_frac = land_frac.rename({'rlat': 'y', 'rlon': 'x'})                       

                        #Get coordinate names
                        if 'longitude' in land_frac.coords:  lat_name1, lon_name1 = 'latitude', 'longitude'
                        elif 'lon' in land_frac.coords:      lat_name1, lon_name1 = 'lat', 'lon'                        
                        if 'rlon' in land_frac.coords:       lat_name2, lon_name2 = 'rlat', 'rlon'
                        elif 'x' in land_frac.coords:        lat_name2, lon_name2 = 'y', 'x'   

                        #Check if coordinates agree and reindex if they are slightly shifted
                        check1a = np.sum(np.abs(land_frac[lat_name1].values - data_exc_ref[lat_name1].values))
                        check1b = np.sum(np.abs(land_frac[lon_name1].values - data_exc_ref[lon_name1].values))
                        check2a = np.sum(np.abs(land_frac[lat_name2].values - data_exc_ref[lat_name2].values))
                        check2b = np.sum(np.abs(land_frac[lon_name2].values - data_exc_ref[lon_name2].values))
                        if (check1a>0.001) or (check1b>0.001):
                            sys.exit('Coordinates between data and LSM do not agree')
                        if (check2a!=0) or (check2b!=0):
                            land_frac[lon_name2] = data_exc_ref[lon_name2]
                            land_frac[lat_name2] = data_exc_ref[lat_name2]                  

                        #Loop over different minimum land fractions
                        for land_min in land_fractions:

                            #Apply land-sea mask
                            data_exc_ref_land = data_exc_ref.where(land_frac>=land_min)
                            data_exc_20y_land = data_exc_20y.where(land_frac>=land_min)

                            #Test that final array still has all entries
                            if data_exc_ref_land.shape!=(3,3):  sys.exit('bla')
                        
                            #Put data in array
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "GWL_level": '1981-2010', "THR_level": THR_level, "land_frac": land_min, 'range_3x3box': 'min'}] = data_exc_ref_land.min()
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "GWL_level": '1981-2010', "THR_level": THR_level, "land_frac": land_min, 'range_3x3box': 'max'}] = data_exc_ref_land.max()
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "GWL_level": GWL_level, "THR_level": THR_level, "land_frac": land_min, 'range_3x3box': 'min'}]   = data_exc_20y_land.min()
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "GWL_level": GWL_level, "THR_level": THR_level, "land_frac": land_min, 'range_3x3box': 'max'}]   = data_exc_20y_land.max()
                    else:
                        
                        #Put data in array
                        data_coll[HSI].loc[{"model": "_".join(model), "city": city, "GWL_level": '1981-2010', "THR_level": THR_level}] = data_exc_ref
                        data_coll[HSI].loc[{"model": "_".join(model), "city": city, "GWL_level": GWL_level, "THR_level": THR_level}]   = data_exc_20y

#Save data in file
fname_out = dir_save + 'HSIs-ThresholdExceedance_' + RCP + '_GWL.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll.to_netcdf(fname_out)    


## Prepare time period data

In [ ]:
#Define output folder
dir_save = dir_out + 'EURO-CORDEX' + out_name + '/'

#Select time periods
time_periods = [[1981, 2010],
                [2036, 2065],
                [2070, 2099]]

#Define time string
time_strings = [str(time[0]) + '-' + str(time[1]) for time in time_periods]

THR_levels = ['Level1', 'Level2', 'Level3', 'Level4']

#Define RCPs
RCP = 'rcp85'
scenarios = ['historical', RCP]

#Define minimum fraction of land required on each grid point
land_fractions = [0, 25, 50, 75, 100]

#Combine model names to one string
mod_names = ["_".join(model) for model in all_models[RCP]]


#Create empty numpy array and define coordinates
if out_name=='_3x3':
    da_empty = np.zeros((len(all_models[RCP]), len(cities), len(time_periods), len(THR_levels), len(land_fractions), 2)) * np.NaN
    coords={'model':        ('model', mod_names),
            'city':         ('city', cities),
            'time_period':  ('time_period', time_strings),
            'THR_level':    ('THR_level', THR_levels),
            'land_frac':    ('land_frac', land_fractions),
            'range_3x3box': ('range_3x3box', ['min', 'max'])}
else:
    da_empty = np.zeros((len(all_models[RCP]), len(cities), len(time_periods), len(THR_levels))) * np.NaN
    coords={'model':       ('model', mod_names),
            'city':        ('city', cities),
            'time_period': ('time_period', time_strings),
            'THR_level':   ('THR_level', THR_levels)}

#Create empty data array for storing SREX-averaged heat indices
data_coll = xr.Dataset(coords=coords)
for HSI in HSI_names:  data_coll[HSI] = (coords.keys(), da_empty.copy())

#Loop over cities
for city in cities:
    
    print(city, end=', ')

    #Loop over models
    for i1, model in enumerate(all_models[RCP]):

        #Get name of CMIP5 driving model
        if ('CNRM-CERFAC' in model[0]) or ('CSIRO-QCCCE' in model[0]) or ('MPI-M' in model[0]) or ('NOAA-GFDL' in model[0]):
            mod_CMIP5 = '-'.join(model[0].split('-')[2:])
        else:
            mod_CMIP5 = '-'.join(model[0].split('-')[1:])
            
        #Loop over scenarios
        create = 1
        for scen in scenarios:

            #Define folder
            dir_data = dir_CORDEX + city + '/' + scen + '/'

            #Read data
            fnames_CORDEX = [file for file in os.listdir(dir_data) if scen + '_' in file and model[0] in file and model[1] in file and model[2] in file]
            if out_name=='':  fnames_CORDEX = [file for file in fnames_CORDEX if 'HSIs_' + city in file]
            else:             fnames_CORDEX = [file for file in fnames_CORDEX if out_name[1::] in file]
            if len(fnames_CORDEX)!=1:  sys.exit('File is not unique')
            data_read = xr.open_dataset(dir_data + fnames_CORDEX[0])

            #Concatenate data
            if create==1:
                data_CORDEX = data_read
                create = 0
            else:
                data_CORDEX = xr.concat((data_CORDEX, data_read), dim='time')            
        
        #Loop over time periods
        for time_period, time_string in zip(time_periods, time_strings):

            #Select data in selected time period
            time_sel = slice(str(time_period[0]), str(time_period[1]))
               
            #Get data in 20y-period
            data_20y = data_CORDEX.sel(time=time_sel)
            
            #Loop over HSIs
            for HSI in HSI_names:
                
                #Correct TN and TX
                if HSI in ['TN', 'TX']:  data_sel = data_20y[HSI] - 273.15
                else:                    data_sel = data_20y[HSI]
                
                #Loop over levels
                for THR_level in THR_levels:
                    
                    #Calculate exceedance
                    data_exc = data_sel>=thresholds[HSI + "_" + THR_level]
                    data_exc = data_exc.sum('time') / (end_year - start_year + 1)
                    
                    if out_name=='_3x3':
                        
                        #Read land fraction for each model and city
                        land_frac = data_LSM[model[0] + '_' + model[1] + '_' + city]

                        #Rename land fraction for MPI-WRF combination
                        if (model[0]=='MPI-M-MPI-ESM-LR') and (model[1]=='IPSL-WRF381P'):
                            land_frac = land_frac.rename({'rlat': 'y', 'rlon': 'x'})                       

                        #Get coordinate names
                        if 'longitude' in land_frac.coords:  lat_name1, lon_name1 = 'latitude', 'longitude'
                        elif 'lon' in land_frac.coords:      lat_name1, lon_name1 = 'lat', 'lon'                        
                        if 'rlon' in land_frac.coords:       lat_name2, lon_name2 = 'rlat', 'rlon'
                        elif 'x' in land_frac.coords:        lat_name2, lon_name2 = 'y', 'x'   

                        #Check if coordinates agree and reindex if they are slightly shifted
                        check1a = np.sum(np.abs(land_frac[lat_name1].values - data_exc[lat_name1].values))
                        check1b = np.sum(np.abs(land_frac[lon_name1].values - data_exc[lon_name1].values))
                        check2a = np.sum(np.abs(land_frac[lat_name2].values - data_exc[lat_name2].values))
                        check2b = np.sum(np.abs(land_frac[lon_name2].values - data_exc[lon_name2].values))
                        if (check1a>0.001) or (check1b>0.001):
                            sys.exit('Coordinates between data and LSM do not agree')
                        if (check2a!=0) or (check2b!=0):
                            land_frac[lon_name2] = data_exc[lon_name2]
                            land_frac[lat_name2] = data_exc[lat_name2]                  

                        #Loop over different minimum land fractions
                        for land_min in land_fractions:

                            #Apply land-sea mask
                            data_exc_land = data_exc.where(land_frac>=land_min)

                            #Test that final array still has all entries
                            if data_exc_land.shape!=(3,3):  sys.exit('bla')

                            #Put data in array
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "time_period": time_string, "THR_level": THR_level, "land_frac": land_min, 'range_3x3box': 'min'}] = data_exc_land.min()
                            data_coll[HSI].loc[{"model": "_".join(model), "city": city, "time_period": time_string, "THR_level": THR_level, "land_frac": land_min, 'range_3x3box': 'max'}] = data_exc_land.max()
                    else:
                        
                        #Put data in array
                        data_coll[HSI].loc[{"model": "_".join(model), "city": city, "time_period": time_string, "THR_level": THR_level}] = data_exc
                        
#Save data in file
fname_out = dir_save + 'HSIs-ThresholdExceedance_' + RCP + '_time-periods.nc'
if os.path.exists(fname_out): os.remove(fname_out)
data_coll.to_netcdf(fname_out)
